In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
156,ABW,North America,Aruba,2020-08-21,1387.0,91.0,70.429,6.0,1.0,0.286,...,7.452,35973.781,NaN,NaN,11.62,NaN,NaN,NaN,NaN,76.29
391,AFG,Asia,Afghanistan,2020-08-21,37856.0,97.0,61.714,1385.0,2.0,3.143,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.5,64.83
544,AGO,Africa,Angola,2020-08-21,2015.0,49.0,36.143,92.0,2.0,1.714,...,1.362,5819.495,NaN,276.045,3.94,NaN,NaN,26.664,NaN,61.15


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-20,France,226821.0,0.0,30508.0,0.0,0.0,0.0
0,2020-08-20,China,89571.0,0.0,4713.0,0.0,0.0,0.0
0,2020-08-20,Italy,255736.0,0.0,35438.0,0.0,0.0,0.0
0,2020-08-20,Spain,375661.0,0.0,28837.0,0.0,0.0,0.0
0,2020-08-20,United States,5616981.0,0.0,174456.0,0.0,0.0,0.0
0,2020-08-20,World,22885782.0,0.0,795286.0,0.0,0.0,0.0
0,2020-08-20,United Kingdom,322329.0,0.0,41445.0,0.0,0.0,0.0
0,2020-08-20,Germany,229352.0,0.0,9268.0,0.0,0.0,0.0
0,2020-08-20,Iran,353024.0,0.0,20293.0,0.0,0.0,0.0
0,2020-08-20,Turkey,254203.0,0.0,6056.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-21,France,226821.0,229814.0,30508.0,30480.0,-2993.0,28.0
0,2020-08-21,China,89571.0,89567.0,4713.0,4709.0,4.0,4.0
0,2020-08-21,Italy,255736.0,256118.0,35438.0,35418.0,-382.0,20.0
0,2020-08-20,Spain,375661.0,377906.0,28837.0,28813.0,-2245.0,24.0
0,2020-08-21,United States,5616981.0,5573847.0,174456.0,174255.0,43134.0,201.0
0,2020-08-21,World,22885782.0,22705645.0,795286.0,794104.0,180137.0,1182.0
0,2020-08-21,United Kingdom,322329.0,322280.0,41445.0,41403.0,49.0,42.0
0,2020-08-21,Germany,229352.0,228621.0,9268.0,9253.0,731.0,15.0
0,2020-08-21,Iran,353024.0,352558.0,20293.0,20264.0,466.0,29.0
0,2020-08-21,Turkey,254203.0,254520.0,6056.0,6058.0,-317.0,-2.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")